## 14.4 Selection sort

Decrease-by-one algorithms split the input collection into one item and
the rest, recursively process the rest and then, if necessary, combine
the subsolution with the removed item.

Non-empty sequences are easily split into their head and tail,
but considering a different split leads to a new sorting algorithm.

### 14.4.1 Recursive version

To split a sequence differently, we ask ourselves what item should be
removed so that the combination step becomes easier.

For a sorting algorithm, a possible answer is: the item with the smallest key.
Once we sorted the remaining items, the combination step just prepends
the smallest item to put it at the start of the sorted sequence.
This is called **selection sort** because at each step we select
the smallest of the still-unsorted items.

Instead of extracting the head and tail of a sequence, we need to extract
the minimum and the rest, without the minimum.
Both auxiliary functions must use the key function to do their job.
Let's call them 'min' and 'without_min'.
The algorithm for selection_sorted(*unsorted*, *key*) is:

1. if *unsorted*  is empty:
   1. let *sorted* be ()
2. otherwise:
   1. let *decreased* be without_min(*unsorted*, *key*)
   1. let *subsolution* be selection_sorted(*decreased*, *key*)
   2. let *sorted* be prepend(min(*unsorted*, *key*), *subsolution*)

This isn't a tail-recursive algorithm, so I'll skip the algorithms
for the auxiliary functions and proceed to an iterative version.

### 14.4.2 Iterative version

Selection sort is another common way to sort cards.
We pick up all dealt cards in one go and fan them in one hand.
With the other hand, we pick the lowest card and put it in the left-most
position of the fan. Then we choose the next lowest card and insert it
in the second position. We repeat this until all cards are sorted.

Like for insertion sort, the fan is divided into a left sorted part that grows
and a right unsorted part that shrinks. In selection sort,
we move the smallest card in the unsorted part to the end of the sorted part.

Both algorithms do two things:

1. choose the next item to process from the unsorted part
2. put it in its correct place in the sorted part.

Insertion sort does less work in step&nbsp;1 (take the next unsorted item)
and more work in step&nbsp;2 (search for the place where to insert the item).
Selection sort does more work in step&nbsp;1 (search for the smallest item) and
less in step&nbsp;2 (append the item to the sorted part).

In a way, insertion and selection sort are the inverse of each other.
Insertion sort knows where the next item to sort is but
doesn't know where it will end up, whereas
selection sort doesn't know where the next item to sort is but
knows where it needs to end up.

The in-place version of selection sort swaps the smallest unsorted item with
the left-most unsorted item. This puts the smallest item in its place,
without shifting any items.

The next table again shows the sorted part in monospaced font.
The first letter S is swapped with G,
the alphabetically first letter in the unsorted part.
Then the second letter O is swapped with I,
the alphabetically first letter in the remaining unsorted part, and so on.

0|1|2|3|4|5|6
-|-|-|-|-|-|-
S|O|R|T|I|N|G
`G`|O|R|T|I|N|S
`G`|`I`|R|T|O|N|S
`G`|`I`|`N`|T|O|R|S
`G`|`I`|`N`|`O`|T|R|S
`G`|`I`|`N`|`O`|`R`|T|S
`G`|`I`|`N`|`O`|`R`|`S`|T

Insertion sort starts with a sorted part with one item and stops when the
unsorted part is empty. Selection sort starts with an empty sorted part,
and stops when the unsorted part has one item. Because selection sort
moves in each step the smallest item to the sorted part,
all unsorted items are larger than all sorted items. After sorting *n*−1 items,
the remaining item is the largest one and is already in the last position.

Selection sort minimises the number of swaps: it does at most *n* − 1&nbsp;swaps
because each item gets into its place with a single swap.
The in-place algorithm is:

1. for each *first unsorted* from 0 to *n* − 2:
   1. let *smallest* be the index of the item with smallest key in *items*[*first unsorted*:*n*]
   2. swap *items*[*first unsorted*] and *items*[*smallest*]

#### Exercise 14.4.1

What's the complexity of in-place selection sort?

_Write your answer here._

[Answer](../32_Answers/Answers_14_4_01.ipynb)

### 14.4.3 Code

The implementation of the algorithm consists of a linear search for the item
with the smallest key, from the first unsorted item onwards,
followed by a swap with the first unsorted item.

In [1]:
from typing import Callable


def selection_sort(items: list, key: Callable) -> None:
    """Sort the items in-place, with keys in non-decreasing order.

    Preconditions: for any indices i and j,
    key(items[i]) and key(items[j]) are comparable
    """
    for first_unsorted in range(0, len(items) - 1):
        smallest = first_unsorted
        for index in range(smallest + 1, len(items)):
            if key(items[index]) < key(items[smallest]):
                smallest = index
        unsorted_item = items[first_unsorted]
        items[first_unsorted] = items[smallest]
        items[smallest] = unsorted_item

Like for insertion sort, we need a version that isn't in-place for testing.

In [2]:
from algoesup import test

%run -i ../m269_sorting


def selection_sorted(unsorted: list, key: Callable) -> list:
    """Return a permutation with keys in non-decreasing order.

    Preconditions: for any indices i and j,
    key(unsorted[i]) and key(unsorted[j]) are comparable
    """
    result = list(unsorted)  # make a copy
    selection_sort(result, key)
    return result


test(selection_sorted, sorting_tests)

Testing selection_sorted...
Tests finished: 7 passed (100%), 0 failed.


Let's check the complexity is always quadratic,
even when the sequence is sorted.

In [3]:
for doubling in range(5):
    items = list(range(100 * 2**doubling))  # sorted order
    %timeit -r 5 selection_sorted(items, identity)

230 μs ± 292 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
906 μs ± 398 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
4.12 ms ± 10.6 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
17.5 ms ± 56.9 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
70.6 ms ± 121 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [4]:
for doubling in range(5):
    items = list(range(100 * 2**doubling, -1, -1))  # reverse order
    %timeit -r 5 selection_sorted(items, identity)

240 μs ± 184 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
938 μs ± 1.06 μs per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
4.26 ms ± 10 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
18.2 ms ± 53.8 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
72.9 ms ± 63.2 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)


The run-times are similar for ascending and descending sequences,
and quadruple as the length doubles.
This confirms the constantly quadratic complexity. (I like alliterations a lot.)

### 14.4.4 Select largest

The selection sort algorithm swaps the *smallest* and *first* unsorted items.
It's also possible to swap the *largest* and *last* unsorted items,
i.e. to move the largest item to the end of the unsorted part.
In this 'mirror' version,
the sorted part is on the right and the unsorted part is on the left.
[This visualisation](https://learn2.open.ac.uk/mod/oucontent/view.php?id=2554725&extra=thumbnail_idm30) shows the approach.

#### Exercise 14.4.2

Redo the example with the word SORTING for this 'select largest' version.
Don't write a table as I did: just write the sequence of letters
as it changes from SORTING to GINORST, one sequence per line,
with a hyphen to separate the right sorted part from the left unsorted part.

SORTING−\
...\
G−INORST

[Hint](../31_Hints/Hints_14_4_02.ipynb)
[Answer](../32_Answers/Answers_14_4_02.ipynb)

⟵ [Previous section](14_3_insertion_sort.ipynb) | [Up](14-introduction.ipynb) | [Next section](14_5_merge_sort.ipynb) ⟶